### Introduction to Python Descriptors

Suppose you have a class Person with two instance attributes first_name and last_name:

In [1]:
class Person:
    
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name

And you want the first_name and last_name attributes to be non-empty strings. These plain attributes cannot guarantee this.

To enforce the data validity, you can use property with a getter and setter methods, like this:

In [2]:
class Person:
    
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name
        
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise ValueError('The first name must be a string')
        
        if len(value) == 0:
            raise ValueError('The first name cannot be empty')
        
        self._first_name = value
    
    @property
    def last_name(self):
        return self._last_name
    
    @last_name.setter
    def last_name(self, value):
        if not isinstance(value, str):
            raise ValueError('The last name must be a string')
            
        if len(value) == 0:
            raise ValueError('The last name cannot be empty')
            
        self._last_name = value

First, define a descriptor class that implements three methods __set_name__, __get__, and __set__:

In [3]:
class RequiredString:
    
    def __set_name__(self, owner, name):
        self.property_name = name
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        
        return instance.__dict__[self.property_name] or None
    
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError(f'The {self.property_name} must be a string')
            
        if len(value) == 0:
            raise ValueError(f'The {self.property_name} cannot be empty')
            
        instance.__dict__[self.property_name] = value
    

Second, use the RequiredString class in the Person class:

In [4]:
class Person:
    
    first_name = RequiredString()
    last_name = RequiredString()

In [5]:
try:
    person = Person()
    person.first_name = ''
except ValueError as e:
    print(e)

The first_name cannot be empty


### Descriptor Protocol

In Python, the descriptor protocol consists of three methods:

- __get__ gets an attribute value
- __set__ sets an attribute value
- __delete__ deletes an attribute

### What is a Descriptor

Descriptors have two types: data descriptor and non-data descriptor.

1. A data descriptor is an object of a class that implements the __set__ and/or __delete__ method.
2. A non-data descriptor is an object that implements the __get__ method only.

### How Descriptors Work

In [6]:
class RequiredString:
    
    def __set_name__(self, owner, name):
        print(f'__set_name__ was called with owner={owner} and name={name}')
        self.property_name = name

    def __get__(self, instance, owner):
        print(f'__get__ was called with instance={instance} and owner={owner}')
        if instance is None:
            return self

        return instance.__dict__[self.property_name] or None

    def __set__(self, instance, value):
        print(f'__set__ was called with instance={instance} and value={value}')

        if not isinstance(value, str):
            raise ValueError(f'The {self.property_name} must a string')

        if len(value) == 0:
            raise ValueError(f'The {self.property_name} cannot be empty')

        instance.__dict__[self.property_name] = value


class Person:
    first_name = RequiredString()
    last_name = RequiredString()

__set_name__ was called with owner=<class '__main__.Person'> and name=first_name
__set_name__ was called with owner=<class '__main__.Person'> and name=last_name


### The __set_name__ Method

In [7]:
first_name = RequiredString()

In [8]:
first_name.__set_name__(Person, 'first_name')

__set_name__ was called with owner=<class '__main__.Person'> and name=first_name


In [ ]:
self.property_name = name

In [10]:
from pprint import pprint

pprint(Person.__dict__)

mappingproxy({'__dict__': <attribute '__dict__' of 'Person' objects>,
              '__doc__': None,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              'first_name': <__main__.RequiredString object at 0x7f896a7ed490>,
              'last_name': <__main__.RequiredString object at 0x7f896a7edeb0>})


### The __set__ Method

In [11]:
def __set__(self, instance, value):
    
    print(f'__set__ was called with instance={instance} and value={value}')

    if not isinstance(value, str):
        raise ValueError(f'The {self.property_name} must be a string')

    if len(value) == 0:
        raise ValueError(f'The {self.property_name} cannot be empty')

    instance.__dict__[self.property_name] = value

In [12]:
person = Person()
person.first_name = 'John'

__set__ was called with instance=<__main__.Person object at 0x7f896a5ddb80> and value=John


In [ ]:
instance.__dict__[self.property_name] = value

In [14]:
person = Person()
print(person.__dict__)  # {}

person.first_name = 'John'
person.last_name = 'Doe'

print(person.__dict__) # {'first_name': 'John', 'last_name': 'Doe'}

{}
__set__ was called with instance=<__main__.Person object at 0x7f896aff7640> and value=John
__set__ was called with instance=<__main__.Person object at 0x7f896aff7640> and value=Doe
{'first_name': 'John', 'last_name': 'Doe'}


### The __get__ Method

In [15]:
def __get__(self, instance, owner):
    
    print(f'__get__ was called with instance={instance} and owner={owner}')
    if instance is None:
        return self

    return instance.__dict__[self.property_name] or None

In [16]:
person = Person()

person.first_name = 'John'
print(person.first_name)

__set__ was called with instance=<__main__.Person object at 0x7f896a76e220> and value=John
__get__ was called with instance=<__main__.Person object at 0x7f896a76e220> and owner=<class '__main__.Person'>
John


In [17]:
print(Person.first_name)

__get__ was called with instance=None and owner=<class '__main__.Person'>


### Summary
- Descriptors are objects of class that implements one of the method in the descriptor protocol including __set__, __get__, __del__